In [1]:
import pandas as pd
import numpy as np

In [2]:
from datasets import Dataset

df = pd.read_table('./sms_spam_collection/SMSSpamCollection', header=None, encoding="UTF-8")
df.columns = ['label', 'text']

d:\Users\270480510\Email-Classification-NLP\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])

In [4]:
ds = Dataset.from_pandas(df)

In [5]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer

model_nm = "bert-base-uncased"
tokz = AutoTokenizer.from_pretrained(model_nm, use_auth_token=False, trust_remote_code=True, local_files_only=False)

d:\Users\270480510\Email-Classification-NLP\env\Lib\site-packages\transformers\models\auto\tokenization_auto.py:898: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [6]:
def tok_func(x): return tokz(x["text"])

In [7]:
tok_ds = ds.map(tok_func, batched=True)

Map: 100%|██████████| 5572/5572 [00:00<00:00, 57263.21 examples/s]


In [8]:
tok_ds = tok_ds.rename_columns({'label':'labels'})

In [24]:
dds = tok_ds.train_test_split(0.25, seed=42)

train_dataset = dds['train']
test_dataset = dds['test']

train_val_split = train_dataset.train_test_split(0.25, seed=42) 

train_dataset = train_val_split['train']
val_dataset = train_val_split['test']

In [10]:
from transformers import TrainingArguments,Trainer

In [11]:
bs = 128
epochs = 10
lr = 0.0001

In [12]:
args = TrainingArguments(
    'outputs',
    learning_rate=lr,
    warmup_steps=500, 
    fp16=True,
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs,
    weight_decay=0.01
)

In [13]:
from scipy.stats import pearsonr

def corr(preds, labels):
    preds = np.argmax(preds, axis=1)
    corr_coef, _ = pearsonr(preds, labels) 
    return corr_coef

def corr_d(eval_pred):
    preds, labels = eval_pred
    return {'pearson': float(corr(preds, labels))} 

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=2)
trainer = Trainer(model, args, train_dataset=train_dataset, eval_dataset=val_dataset,
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
D:\Users\270480510\AppData\Local\Temp\ipykernel_25588\1647062213.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=train_dataset, eval_dataset=val_dataset,


In [15]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=250, training_loss=0.12745635986328124, metrics={'train_runtime': 146.5305, 'train_samples_per_second': 213.88, 'train_steps_per_second': 1.706, 'total_flos': 1713134581445040.0, 'train_loss': 0.12745635986328124, 'epoch': 10.0})